In [86]:
# number of CPUs to be used
NCPU=20

## Load the dataset

In [87]:
import json, gzip
import numpy as np

aa2idx = {'A':0, 'R':1, 'N':2, 'D':3, 'C':4, 'Q':5, 'E':6, 'G':7, 'H':8, 'I':9,
          'L':10, 'K':11, 'M':12, 'F':13, 'P':14, 'S':15, 'T':16, 'W':17, 'Y':18, 'V':19}

# read .json file
with gzip.open('../data/phipsi.json.gz', 'rb') as f:
    dataset = json.load(f)

# reduse dataset to a list for simpler access
dataset = dataset['phipsi10882']

# convert data to numpy arrays skipping first and last residues
for item in dataset:
    n = len(item['sequence'])
    item['sequence'] = np.array([aa2idx[aa] for aa in item['sequence'][1:n-1]], dtype=np.int8)
    item['phi'] = np.array(item['phi'], dtype=np.float32)[1:n-1]
    item['psi'] = np.array(item['psi'], dtype=np.float32)[1:n-1]
    
    # convert (phi,psi) to their sin() and cos()
    # (4 numbers per angle pair)
    item['avec'] = np.vstack([
        np.sin(item['phi']).T,
        np.cos(item['phi']).T,
        np.sin(item['psi']).T,
        np.cos(item['psi']).T ]).T

## Train / test split

In [88]:
from sklearn.model_selection import train_test_split

# 90% train, 10% test
train,test = train_test_split(dataset, test_size=0.1, random_state=42)

print("Train size: {} proteins".format(len(train)))
print("Test size: {} proteins".format(len(test)))


Train size: 9793 proteins
Test size: 1089 proteins


## K-means clusters from angle vectors

We now perform clustering, but on the train set only. Test set will be used further to assess and tune the number of clusters.

In [89]:
%%time
from sklearn.cluster import KMeans

# combine all proteins in the training set
# into one vector X
X = np.vstack([item['avec'] for item in train])

# split into 20 clusters
kmeans = KMeans(n_clusters=20, max_iter=5, n_jobs=NCPU).fit(X)

CPU times: user 1.63 s, sys: 2.91 s, total: 4.54 s
Wall time: 29.4 s


We now discretize dihedrals (&phi;,&psi;) in both train and test sets based on the clustering results.

In [90]:
# save cluster IDs for both train and test sets
for subset in (train, test):
    for item in subset:
        item['abin'] = np.array(kmeans.predict(item['avec']), dtype=np.int8)

## Split sequences into chunks of equal length

In [91]:
def set_window(train, test, WINDOW):
    for subset in (train, test):
        for item in subset:
            l = len(item['sequence'])

            abin = item['abin']
            seq = item['sequence']

            # for every window, pick the element in the middle and
            # save corresponding dihedral cluster ID in item['Y']
            item['Y'] = np.hstack([item[WINDOW//2] 
                                   for shift in range(0,WINDOW,1) 
                                   for item in np.split(abin[shift:],range(0,l,WINDOW)) 
                                   if len(item) == WINDOW])

            # use 1-hot encoding for every sequence chunk
            seq_chunks = np.vstack([item for shift in range(0,WINDOW,1) 
                                    for item in np.split(seq[shift:],range(0,l,WINDOW)) 
                                    if len(item) == WINDOW])
            item['X'] = np.array(np.eye(20)[seq_chunks], dtype=np.int8).reshape((seq_chunks.shape[0],-1))

Test ```set_window(..)``` function

In [93]:
%%time

WINDOW = 13

set_window(train, test, WINDOW)

item = test[42]
print("Sequence length: {}".format(len(item['sequence'])))
print("Number of {}-mers: {}".format(WINDOW, item['X'].shape[0]))
print("Number of features: {}".format(item['X'].shape[1]))
print("Length of Y: {}".format(item['Y'].shape[0]))

Sequence length: 80
Number of 13-mers: 68
Number of features: 260
Length of Y: 68
CPU times: user 37.6 s, sys: 316 ms, total: 38 s
Wall time: 37.9 s


## Logistic regression


Stack X and Y vectors from all proteins together to create single train/test vectors

In [94]:
X_train = np.vstack([item['X'] for item in train])
Y_train = np.hstack([item['Y'] for item in train])

X_test = np.vstack([item['X'] for item in test])
Y_test = np.hstack([item['Y'] for item in test])

print("X_train shape: {}".format(X_train.shape))
print("Y_train shape: {}".format(Y_train.shape))

X_train shape: (2027748, 260)
Y_train shape: (2027748,)


Applying regular ```sklearn```'s ```LogisticRegression``` to the entire training set is quite time- and memory- consuming. A workaround is mini batch logistic regression accessible via ```SGDClassifier``` class with ```loss``` parameter set to ```'log'```

In [95]:
%%time
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(max_iter=1000, tol=1e-3, loss='log', n_jobs=NCPU)

sgd.fit(X_train, Y_train)

CPU times: user 8min 22s, sys: 5.32 s, total: 8min 28s
Wall time: 31.4 s


In [96]:
print("Train/test scores: {} / {}".format(sgd.score(X_train, Y_train), sgd.score(X_test, Y_test)))

Train/test scores: 0.2580946942124958 / 0.257347556402587


Convert predictions back to angles and plot

In [97]:
cent = kmeans.cluster_centers_
pred = sgd.predict_proba(X_test)[12342].reshape(20,1)
print(np.sum(pred * cent, axis=0), Y_test[12342])
#print(sgd.predict_proba(X_test)[12342].reshape(20,1))
#print(sgd.predict_proba(X_test)[12342].reshape(20,1) * kmeans.cluster_centers_)

[-0.84557816  0.12254535 -0.05796351  0.13275568] 14


## Analyze results

In [104]:
from sklearn.metrics import log_loss

#Yref = kmeans.predict(np.vstack([item['Y'] for item in test]))
Y_test_pred = sgd.predict_proba(X_test)
print(Y_test.shape, Y_test_pred.shape)


(226674,) (226674, 20)


array([ 1, 13,  7, ...,  6,  9,  0], dtype=int8)

In [105]:
log_loss(Y_test,Y_test_pred)

2.2648874212258696